In [8]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import requests
import json
from openai import OpenAI
import torch
import numpy as np
import scipy
import nltk
nltk.download('punkt')
import typing
import MindMap
from transformers import GPT2Tokenizer
import os
import glob
import pandas as pd


[nltk_data] Downloading package punkt to
[nltk_data]     /home/wangqianyue/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [12]:
def confict_triple_ratio(title:str):
    pathdf="/mnt/cephfs/home/wangqianyue/LTG/MindMap-main/data/"+title+"/"
    
    csv_files = glob.glob(os.path.join(pathdf, '*.csv'))
    #创建一个新dataframe，列名为time,subject,predicate,object
    dft=pd.DataFrame(columns=["head","relation","tail","time"])

    for file in csv_files:
        #从file中读取dataframe,忽略行index
        df=pd.read_csv(file)
        #将df拼接至dft中
        dft=pd.concat([dft,df],ignore_index=True)
    print(dft)
    res=MindMap.confict_detection(dft)
    return res

confict_triple_ratio("0424")

                head                relation                     tail time
0              story               is set in               inner city    1
1         inner city              located in  large metropolitan area    1
2      Gary Saunders                      is              teenage boy    1
3      Gary Saunders                lives in               inner city    1
4      Shannon Doyle                      is              young woman    1
..               ...                     ...                      ...  ...
128  Diane's journey  towards self-discovery                    begin    5
129          journey            fraught with               challenges    5
130          journey            fraught with             complexities    5
131            Diane                 embrace   journey with open arms    5
132          chapter                     end                  promise    5

[133 rows x 4 columns]
([[('Diane', 'feels', 'weight of decisions', 3), ('Diane', 'feels', 'weight 

1.0

In [15]:
#对比方法,DOC,KG 冲突jiance 
import MindMap
cpath="/mnt/cephfs/home/wangqianyue/LTG/MindMap-main/data/compare/"

#读取文件中的内容
data=""
title="0"
path=cpath+title+'/'+title+".txt"
with open(path, 'r') as f:
    data +=f.read()

# # #按照\n划分内容
# data=data.split("\n")
# #依次处理data中的内容
# #构建一个空dataframe
# df=pd.DataFrame(columns=["head","relation","tail","time"])
# time=1
# for i in range(len(data)):
#     #如果data[i]不为空
#     if data[i]!="":
        
#         res=MindMap.get_inputkg(data[i],title)
        
#         res["time"]=time
#         time+=1
#         #将res加入dataframe中
#         df=pd.concat([df,res],ignore_index=True)
       
#         #保存dataframe
#         dfpath=cpath+title+'/'+title+".csv"
#         df.to_csv(dfpath,index=False)
#     print(i)
# #删除df中包含控制的行
# df=df.dropna()

#先要抽取KG
#读取dataframe中的内容
dfpath=cpath+title+'/'+title+".csv"
df=pd.read_csv(dfpath)


#然后对比
res=MindMap.confict_detection(df)
print(res)




([[('Lily', 'lives in', 'quaint rural town', 4), ('Lily', 'lives in', 'quaint rural town', 7), ('Lily', 'lives in', 'quaint rural town', 16), ('Lily', 'lives in', 'quaint rural town', 21), ('Lily', 'lives in', 'quaint rural town', 27)], [('quaint rural town', 'surrounded by', 'lush greenery', 4), ('quaint rural town', 'surrounded by', 'lush greenery', 5), ('quaint rural town', 'surrounded by', 'lush greenery', 7), ('quaint rural town', 'surrounded by', 'lush greenery', 12), ('quaint rural town', 'surrounded by', 'lush greenery', 14), ('quaint rural town', 'surrounded by', 'lush greenery', 16), ('quaint rural town', 'surrounded by', 'lush greenery', 18), ('quaint rural town', 'surrounded by', 'lush greenery', 19), ('quaint rural town', 'surrounded by', 'lush greenery', 21), ('quaint rural town', 'surrounded by', 'lush greenery', 27)], [('quaint rural town', 'surrounded by', 'rolling hills', 4), ('quaint rural town', 'surrounded by', 'rolling hills', 5), ('quaint rural town', 'surrounded

In [9]:
def cal_token(text):
    
    tokenizer = GPT2Tokenizer.from_pretrained('/mnt/cephfs/home/wangqianyue/LTG/ppl/gpt2')
    if tokenizer.pad_token is None:
        tokenizer.add_special_tokens({'pad_token': '[PAD]'})
        model.resize_token_embeddings(len(tokenizer))
    tokens = tokenizer.tokenize(text)
    token_count = len(tokens)
    return token_count

def n_gram_entropy(gen_texts, agg="arith"):
    assert agg in ["arith", "geom"]

    return (scipy.stats.mstats.gmean if agg == "geom" else np.mean)(
        [compute_n_gram_entropy(txt) for txt in gen_texts]
    ).item()

def compute_freq(sentence, n=2):
    tokens = nltk.word_tokenize(sentence)
    ngrams = nltk.ngrams(tokens, n)
    return nltk.FreqDist(ngrams)
def compute_n_gram_entropy(sentence, ns=None, weights=None, agg="arith"):
    if ns is None:
        ns = [2, 3]
    if weights is None:
        weights = [2 / 3, 4 / 3]
    assert agg in ["arith", "geom"]

    entropy_list = []
    for n in ns:
        fdist = compute_freq(sentence, n)
        freqs = np.array([freq for _, freq in fdist.items()])
        freqs = freqs / freqs.sum()

        entropy_list.append(np.sum(-freqs * np.log(freqs) / np.log(2)))

    entropy_list = np.array(entropy_list) * np.array(weights)

    return (scipy.stats.mstats.gmean if agg == "geom" else np.mean)(entropy_list)

def cal_generation_quality(text):
    
    gen_texts = [text]
    ngram_entropy = n_gram_entropy(gen_texts)
    ret = {"ngram_entropy": ngram_entropy}
    return ret

#计算generation quality

In [13]:
cpath="/mnt/cephfs/home/wangqianyue/LTG/doc_outputs/stories/doc/1.txt"
#读取文件中的内容
data=""
# title="0"
# path=cpath+title+'/'+title+".txt"
with open(cpath, 'r') as f:
    data +=f.read()
#一个超长string
print(cal_generation_quality(data))

{'ngram_entropy': 11.231971861010939}


In [11]:
cpath="/mnt/cephfs/home/wangqianyue/LTG/MindMap-main/lzp_test/0429/22_15_22_story_e.txt"
#读取文件中的内容
data=""
title="1"
path=cpath
with open(path, 'r') as f:
    data +=f.read()
#一个超长string
print(cal_generation_quality(data))

{'ngram_entropy': 10.214925427419814}


{'ngram_entropy': 10.90322583465683}


In [14]:
#计算PPL
def cal_ppl(text):
    tokenizer = GPT2Tokenizer.from_pretrained('/mnt/cephfs/home/wangqianyue/LTG/MindMap-main/pipline/gpt2/gpt2')
    model = GPT2LMHeadModel.from_pretrained('/mnt/cephfs/home/wangqianyue/LTG/MindMap-main/pipline/gpt2/gpt2')
    tokenizer.pad_token = tokenizer.eos_token
    inputs = tokenizer(text, return_tensors="pt", max_length=1024, truncation=True, padding=True)
    input_ids = inputs.input_ids
    attention_mask = inputs.attention_mask
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)
    loss = outputs.loss
    ppl = torch.exp(loss)
    return ppl.item()
print(cal_ppl(data))


11.610363006591797


In [ ]:
#计算相关性（目前是计算一篇的）/可以改为计算多篇

In [ ]:
from prompt import PROMPT_TEMPLATE
import pickle


#DOC
path_text="/mnt/cephfs/home/wangqianyue/LTG/MindMap-main/data/doc_outputs/annotations/detailed_relevance/doc_docnocontrol_detailedrelevance.csv"
title="3"
df = pd.read_csv(path)
#找出df中id列中，包含doc/3_的行
tag='doc/'+title+'_'
df = df[df['id'].str.contains(tag)]
print(df)
#将df中的内容按照id最后的数字进行排序
df['id'] = df['id'].str.split('_').str[-1].astype(int)
df= df.sort_values('id')
event=df['event'].tolist()
passage=df['passage'].tolist()
ress=[]
explains=[]
for e,p in zip(event,passage):
    
    #分别抽取KG
    e_kg=MindMap.get_inputkg(e,'event')
    p_kg=MindMap.get_inputkg(p,'passage')
    
    result_schema = ResponseSchema(name="result",
                             description="Choose between 'yes' and 'no' to indicate whether the event occurs in its entirety within the passage.")
    explain_schema = ResponseSchema(name="explain",
                                        description="Was a string that describes the reason for the result.")

    response_schemas = [result_schema,
                        explain_schema]

    output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
    format_instructions = output_parser.get_format_instructions()
   
    relevance_prompt_t=PROMPT_TEMPLATE['relevance']
    relevance_prompt_tempate =ChatPromptTemplate.from_template(template=relevance_prompt_t)
    ps=relevance_prompt_tempate.format_messages(event=e,passage=p,format=format_instructions)
    res= chat(ps[0].content)
    
    result=data_process_json(res)['result']
    explain=data_process_json(res)['explain']
    ress.append(result)
    explains.append(explain)

#找出ress中的yes和no的数量
yes=ress.count('yes')
no=ress.count('no')
#计算相关性
relevance=yes/(yes+no)
print(relevance)

保存评估结果
#保存list:ress和explains
path="/mnt/cephfs/home/wangqianyue/LTG/MindMap-main/data/doc_outputs/annotations/"+title+'_'+"ress.pkl"
pathe="/mnt/cephfs/home/wangqianyue/LTG/MindMap-main/data/doc_outputs/annotations/"+title+'_'+"explains.pkl"
# 保存列表
with open(path, 'wb') as f:
    pickle.dump(ress, f)

with open(pathe, 'wb') as f:
    pickle.dump(explains, f)







In [1]:
#针对本文方法
path="/mnt/cephfs/home/wangqianyue/LTG/MindMap-main/data/test1/outline.json"
import json
with open(path, 'r') as f:
    data = json.load(f)
print(data)
    

JSONDecodeError: Expecting ',' delimiter: line 2 column 3 (char 18)

[['Daisy Mayberry', 'walks around', 'her house'], ['Daisy Mayberry', 'mentally prepares', 'herself'], ['Daisy Mayberry', 'brushes', 'her hair'], ['Daisy Mayberry', 'puts on', 'some makeup'], ['Daisy Mayberry', 'looks at', 'herself'], ['Daisy Mayberry', 'runs hand over', 'her face'], ['Daisy Mayberry', 'makes a fist with', 'her lips'], ['thought', 'occurs to', 'Daisy Mayberry'], ['Daisy Mayberry', 'realizes', 'she is no longer a young woman'], ['Daisy Mayberry', 'remembers', "she isn't just any old middle-aged woman"], ['Daisy Mayberry', 'learns', 'she has cancer'], ['doctors', 'can do nothing for', 'Daisy Mayberry'], ['Daisy', 'received', 'news'], ['prognosis', 'was', 'not good'], ['doctor', 'told', 'Daisy'], ['doctor', 'advised', 'Daisy'], ['Daisy', 'should spend', 'rest of her days'], ['Daisy', 'should spend', 'with loved ones'], ['Daisy Mayberry', 'heard', 'words'], ['Daisy Mayberry', 'understands', 'going to die'], ['Daisy Mayberry', 'feels', "never feel warmth of another living pe

In [ ]:
#计算实体重叠度
